# Demo: Radio Telescope

In [ ]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from math import acos, sin, cos, radians, atan
from ortools.constraint_solver import pywrapcp
import plotly.express as px
import plotly.graph_objects as go
from plotly.validators.scatter.marker import SymbolValidator

### Load Data

In [ ]:
# load dataframe
data = pd.read_csv('data-demo/hygdata_v3.csv').set_index('id') # read a CSV file
data = data[~data['proper'].isna()][['proper', 'dist', 'ra', 'dec', 'pmra', 'pmdec']].dropna()
data = data.reset_index()

# unit conversions
data['dist'] = data['dist'].apply(lambda x: x*3.262) # convert from parsecs to lightyears
data['ra'] = data['ra'].apply(lambda x: x*15) # convert from hours to degrees 
data['pmra'] = data['pmra'].apply(lambda x: x*2.7777776630942*10**(-7)) # convert to degrees per year
data['pmdec'] = data['pmdec'].apply(lambda x: x*2.7777776630942*10**(-7)) # convert to degrees per year

# rename fields
data = data.rename(columns={'ra' : 'ra_2000', 'dec' : 'dec_2000'})

# calcualte 2020 right ascension and declination
data['ra_2020'] = data['ra_2000'] + 20*data['pmra']
data['dec_2020'] = data['dec_2000'] + 20*data['pmdec']

data.head() # preview

### Create and Solve Model

In [ ]:
# Create a distance matrix (great-circle distance on unit circle)

# parameters
alpha = 1 
beta = 0

# get normal vectors
theta = data['ra_2020'].apply(lambda x: radians(x)).to_dict()
phi = data['dec_2020'].apply(lambda x: radians(x+90)).to_dict()
dist = data['dist'].apply(lambda x: radians(x)).to_dict()
n = {}

for i in range(len(data)):
    x = cos(theta[i])*sin(phi[i])
    y = sin(theta[i])*sin(phi[i])
    z = cos(phi[i])
    n[i] = np.array([x,y,z])
    
d = np.zeros((len(data),len(data)))
for i in range(len(data)):
    for j in range(len(data)): 
        if i < j:
            rot_dist = atan(np.linalg.norm(np.cross(n[i],n[j]))/np.dot(n[i],n[j]))
            zoom_dist = abs(dist[i] - dist[j])
            total = alpha*rot_dist + beta*zoom_dist
            d[i,j] = total
            d[j,i] = total

In [ ]:
def TSP(d, s, t):
    """Return optimal tour from s to t.
    
    Args:
        d (np.ndarray): Distance matrix.
        s (int): start index.
        t (int): end index.
    """
    # number of locations, number of vehicles, start location
    manager = pywrapcp.RoutingIndexManager(len(d), 1, [s], [t])
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return d[from_node, to_node]*10000

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    def get_routes(solution, routing, manager):
        """Get vehicle routes from a solution and store them in an array."""
        routes = []
        for route_nbr in range(routing.vehicles()):
            index = routing.Start(route_nbr)
            route = [manager.IndexToNode(index)]
            while not routing.IsEnd(index):
                index = solution.Value(routing.NextVar(index))
                route.append(manager.IndexToNode(index))
            routes.append(route)
        return routes

    solution = routing.Solve()
    print('Objective value:',solution.ObjectiveValue()/10000)
    return get_routes(solution, routing, manager)[0]

In [ ]:
tour = TSP(d,0,0)

In [ ]:
# feasible tour subject to the time window constraints
f_tour = list(data.sort_values('ra_2020').index)
f_tour.append(f_tour[0])
print('Objective value:',sum(d[f_tour[i],f_tour[i+1]] for i in range(len(f_tour)-1)))

# improve feasible solution

def improve(tour, l, r):
    subset = tour[l:r]
    d_tmp = d[subset,np.array([[i] for i in subset])]
    subset_reopt = TSP(d_tmp,0,len(d_tmp)-1)
    subset_reopt = [subset[i] for i in subset_reopt]
    tour[l:r] = subset_reopt
    return tour

# round 1
window = 10
shift = 1
for i in range(0,len(f_tour)-window,shift):
    f_tour = improve(f_tour, i, i + window)

# round 2
window = 30
shift = 5
for i in range(0,len(f_tour)-80,shift):
    f_tour = improve(f_tour, i, i + window)
    
window = 15
shift = 3
for i in range(len(f_tour)-80,len(f_tour)-15,shift):
    f_tour = improve(f_tour, i, i + window)
    
# round 3
f_tour = improve(f_tour, f_tour.index(105), f_tour.index(126)+1)
    
print('Objective value:',sum(d[f_tour[i],f_tour[i+1]] for i in range(len(f_tour)-1)))

In [ ]:
tour = f_tour

In [ ]:
obj_val = sum(d[tour[i],tour[i+1]] for i in range(len(tour)-1))

### Visualizations

In [ ]:
# generate a visualization of the star locations
plt.figure(figsize=(20,10))
ax = plt.axes()
ax.set_facecolor('black')
plt.scatter(data.ra_2020,data.dec_2020, marker='*', color='white', s=70)
for index, row in data.iterrows():
    plt.annotate(row['proper'], 
                 (row['ra_2020'],row['dec_2020']),
                 textcoords="offset points", 
                 xytext=(5,10),
                 color= 'white',
                 ha='center')
plt.title('The Locations of 146 Stars', fontsize= 20)
plt.xlabel('Right Ascension (Degrees)', fontsize= 15)
plt.ylabel('Declination (Degrees)', fontsize= 15)
plt.show()

In [ ]:
# Generate scatter plot of star locations
plt.figure(figsize=(20,10))
ax = plt.axes()
ax.set_facecolor('black')
plt.scatter(data.ra_2020,data.dec_2020, marker='*', color='white', s=70)
for index, row in data.iterrows():
    plt.annotate(row['proper'], 
                 (row['ra_2020'],row['dec_2020']),
                 textcoords="offset points", 
                 xytext=(5,10),
                 color= 'white',
                 ha='center')
    
# Add path through these stars
tour_proper = [data.iloc[i].proper for i in tour]
tour_proper.append(tour_proper[0])
tour_ra = [data[data.proper == proper].ra_2020.to_list()[0] for proper in tour_proper]
tour_dec = [data[data.proper == proper].dec_2020.to_list()[0] for proper in tour_proper]
plt.plot(tour_ra, tour_dec, color='white')

# Set axes titles and display
plt.title('A Tour of 146 Stars', fontsize= 20)
plt.xlabel('Right Ascension (Degrees)', fontsize= 15)
plt.ylabel('Declination (Degrees)', fontsize= 15)
plt.show()

### Create Web Example

In [ ]:
tour = pd.DataFrame().assign(name = tour_proper,
                             ra = tour_ra,
                             dec = tour_dec)
tour = tour.reset_index()

In [ ]:
# get distance from star i to i + 1, true if dist between stars inner; false if outer
delta_ra = []
inner_ra = []
delta_dec = []
inner_dec = []

for i in range(len(tour)):
    if i+1 == len(tour):
        delta_ra.append(0.0)
        inner_ra.append(True)
        delta_dec.append(0.0)
        inner_dec.append(True)
    else:
        tmp = abs(tour.loc[i+1]['ra'] - tour.loc[i]['ra'])
        delta_ra.append(min(tmp, 360 - tmp))
        inner_ra.append(tmp == min(tmp, 360 - tmp))
        tmp = abs(tour.loc[i+1]['dec'] - tour.loc[i]['dec'])
        delta_dec.append(min(tmp, 180 - tmp))
        inner_dec.append(tmp == min(tmp, 180 - tmp))
    
tour = tour.assign(delta_ra = delta_ra,
                   inner_ra = inner_ra,
                   delta_dec = delta_dec,
                   inner_dec = inner_dec) 

tour['total_dist'] = tour.apply(lambda x: math.sqrt(x.delta_ra**2 + x.delta_dec**2), axis=1)

In [ ]:
# parameters
day = 180 # ticks
delay = 0.25 # look at each star for 0.25 ticks
rate = 0.0025

In [ ]:
loc = []
loc.append((0,0))
current_star = 0
departure_time = delay
arrival_time = departure_time + rate*tour.loc[current_star]['total_dist']
t = 1
while(current_star < len(tour) - 1):
    if t <= departure_time:
        loc.append(loc[t-1])
    else:
        if t >= arrival_time:
            current_star += 1
            departure_time = arrival_time + delay
            arrival_time = departure_time + rate*tour.loc[current_star]['total_dist']
            loc.append((tour.loc[current_star]['ra'],tour.loc[current_star]['dec']))
        else:
            dist_to_travel = tour.loc[current_star]['total_dist']
            dist_travalled = (t - departure_time)/rate
            pct_tavalled = dist_travalled / dist_to_travel
            s_ra = tour.loc[current_star]['ra']
            t_ra = tour.loc[current_star+1]['ra']
            if tour.loc[current_star]['inner_ra']:
                if s_ra <= t_ra:
                    ra = s_ra + (t_ra - s_ra)*pct_tavalled
                else:
                    ra = s_ra - (s_ra - t_ra)*pct_tavalled
            else:
                if s_ra <= t_ra:
                    tmp = s_ra - (360 - abs(t_ra - s_ra))*pct_tavalled
                    ra = tmp if tmp > 0 else tmp + 360
                else:
                    tmp = s_ra + (360 - abs(t_ra - s_ra))*pct_tavalled
                    ra = tmp if tmp <= 360 else tmp - 360

            s_dec = tour.loc[current_star]['dec']
            t_dec = tour.loc[current_star+1]['dec']
            if tour.loc[current_star]['inner_dec']:
                if s_dec <= t_dec:
                    dec = s_dec + (t_dec - s_dec)*pct_tavalled
                else:
                    dec = s_dec - (s_dec - t_dec)*pct_tavalled
            else:
                if s_dec <= t_dec:
                    tmp = s_dec - (180 - abs(t_dec - s_dec))*pct_tavalled
                    dec = tmp if tmp > -90 else tmp + 180
                else:
                    tmp = s_dec + (180 - abs(t_dec - s_dec))*pct_tavalled
                    dec = tmp if tmp <= 90 else tmp - 180     
            loc.append((ra,dec)) 
    t += 1

In [ ]:
x,y = zip(*loc)
df = pd.DataFrame().assign(x=x,y=y).reset_index().rename(columns={'index' : 't'})
fig = px.scatter(df, x="x", y="y", 
                 animation_frame="t", range_x=[0,360], range_y=[-90,90], 
                 title='Improved Feasible Telescope Tour of 146 Stars  (Obj. Value: %f)' % (obj_val))
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 200 #200
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 500 #500
tmp = px.line(tour, x="ra", y="dec",hover_data=['index','name']).data[0]
tmp.mode = 'lines+markers'
fig.add_trace(tmp)

day_regions = []

for i in range(day):
    l = (90 + (360.0/day)*i) % 360
    r = (270 + (360.0/day)*i) % 360
    if l%360 < r%360:
        x=[l,l,r,r,l]
        y=[90,-90,-90,90,90]             
    else:
        x=[l,l,360,360,l,None,0,0,r,r,0]
        y=[90,-90,-90,90,90,None,90,-90,-90,90,90]
        
    region = go.Scatter(x=x, 
                        y=y, 
                        fill="toself", 
                        opacity=0.4, 
                        fillcolor='#FFFFFF', 
                        marker = dict(size=0.1), 
                        line = dict(color='#FFFFFF'), 
                        visible = False,
                        showlegend = False)
    day_regions.append(region)
    fig.add_trace(region)
    
fig.data[1].line.color = '#FFFFFF'
fig.data[1].marker.size = 5
fig.data[0].marker.color = '#000000'
fig.data[0].marker.size = 15
fig.update_xaxes(nticks=4, title='Right Ascension (Degrees)')
fig.update_yaxes(nticks=4, title='Declination (Degrees)')
fig.layout.plot_bgcolor= '#000000'

for i in range(len(fig.frames)):
    fig.frames[i].data[0].marker.color = '#F4E318'
    tmp = list(fig.frames[i].data)
    stars = px.line(tour, x="ra", y="dec",hover_data=['name']).data[0]
    stars.marker.color = '#FFFFFF'
    stars.marker.size = 5
    stars.mode = 'lines+markers'
    stars.line.color = '#FFFFFF'
    tmp.append(stars)
    reg = day_regions[i]
    reg.visible = True
    tmp.append(reg)
    tmp = tuple(tmp)
    fig.frames[i].data = tmp

In [ ]:
fig